# XGBoost 

In [0]:
import pandas as pd
import numpy as np
import os

import json

from sklearn.model_selection  import train_test_split

In [0]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'


# 훈련 데이터 가져오는 부분이다.
train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [0]:

train_input = np.stack((train_q1_data, train_q2_data), axis=1)

In [4]:
print(train_input.shape)

(298526, 2, 31)


In [11]:
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

ValueError: ignored

In [12]:
train_input

array([[[  2,   3,   1, ...,   0,   0,   0],
        [  4,   3,   1, ...,   0,   0,   0]],

       [[  2,  11,   1, ...,   0,   0,   0],
        [  2,   3,   1, ...,   0,   0,   0]],

       [[  4,   9,  84, ...,   0,   0,   0],
        [  4,  13,   5, ...,   0,   0,   0]],

       ...,

       [[  4,  21,  19, ...,   0,   0,   0],
        [  4,  21,  19, ...,   0,   0,   0]],

       [[  2,  11,   1, ...,   0,   0,   0],
        [  2,  11, 169, ...,   0,   0,   0]],

       [[  4,   9,   5, ...,   0,   0,   0],
        [  4,   9,   5, ...,   0,   0,   0]]], dtype=int32)

In [0]:
import xgboost as xgb

In [0]:
train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label) # 학습 데이터 읽어 오기
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label) # 평가 데이터 읽어 오기

data_list = [(train_data, 'train'), (eval_data, 'valid')]

### 53분 시작.

In [15]:
params = {} # 인자를 통해 XGB모델에 넣어 주자 
params['objective'] = 'binary:logistic' # 로지스틱 예측을 통해서 
params['eval_metric'] = 'rmse' # root mean square error를 사용  

bst = xgb.train(params, train_data, num_boost_round = 1000, evals = data_list, early_stopping_rounds=10)

[0]	train-rmse:0.483811	valid-rmse:0.48421
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[1]	train-rmse:0.473357	valid-rmse:0.474323
[2]	train-rmse:0.466958	valid-rmse:0.468227
[3]	train-rmse:0.462158	valid-rmse:0.463641
[4]	train-rmse:0.458526	valid-rmse:0.46025
[5]	train-rmse:0.455729	valid-rmse:0.457584
[6]	train-rmse:0.453551	valid-rmse:0.455493
[7]	train-rmse:0.451809	valid-rmse:0.453967
[8]	train-rmse:0.450433	valid-rmse:0.452798
[9]	train-rmse:0.447502	valid-rmse:0.450016
[10]	train-rmse:0.446145	valid-rmse:0.448812
[11]	train-rmse:0.445218	valid-rmse:0.448059
[12]	train-rmse:0.44469	valid-rmse:0.447688
[13]	train-rmse:0.443902	valid-rmse:0.447063
[14]	train-rmse:0.442311	valid-rmse:0.445667
[15]	train-rmse:0.441441	valid-rmse:0.44491
[16]	train-rmse:0.440906	valid-rmse:0.444438
[17]	train-rmse:0.440072	valid-rmse:0.443853
[18]	train-rmse:0.439715	valid-rmse:0.443573
[19]	train-rms

In [0]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

In [0]:
test_input = np.stack((test_q1_data, test_q2_data), axis=1) 
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [0]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv('/content/gdrive/My Drive/Colab Notebooks/Data/simple_xgb.csv', index=False)

In [19]:
!ls -al "/content/gdrive/My Drive/Colab Notebooks/Data/"

total 245022
-rw------- 1 root root      1449 May 19 17:18 ChatBotData.csv_short
-rw------- 1 root root  62700340 Jun 21 04:21 rnn_predict.csv
-rw------- 1 root root   5190322 May 12 08:17 sample_submission.csv.zip
-rw------- 1 root root  42874600 Jun 21 04:42 simple_xgb.csv
-rw------- 1 root root 117931806 May 12 08:18 test.csv.zip
-rw------- 1 root root  22202741 May 12 08:17 train.csv.zip
